# Lab Three: Extending Logistic Regression
 

#### Everett Cienkus, Blake Miller, Colin Weil

### 1. Preparation and Overview

#### 1.1 Business Case

Explain the task and what business-case or use-case it is designed to solve (or designed to investigate). Detail exactly what the classification task is and what parties would be interested in the results. For example, would the model be deployed or used mostly for offline analysis? 

#### 1.2 Preparation of Data

In [78]:
import pandas as pd
import numpy as np

# Define and prepare your class variables.
df = pd.read_csv('wine_dataset/winequality-red.csv')
X = df.drop(columns = ['quality'])
y = df['quality']
# Use proper variable representations (int, float, one-hot, etc.).
# Use pre-processing methods (as needed) for dimensionality reduction, 
# scaling, etc. Remove variables that are not needed/useful for the analysis. 
# Describe the final dataset that is used for classification/regression
display(X.info())
display(y.info())
# (include a description of any newly formed variables you created).
# MAKE SURE TO NORMALIZE VALUES

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
dtypes: float64(11)
memory usage: 137.5 KB


None

<class 'pandas.core.series.Series'>
RangeIndex: 1599 entries, 0 to 1598
Series name: quality
Non-Null Count  Dtype
--------------  -----
1599 non-null   int64
dtypes: int64(1)
memory usage: 12.6 KB


None

#### 1.3 Division of Trainig and Testing Data

In [79]:
# Divide your data into training and testing data using an 80% training 
# and 20% testing split. Use the cross validation modules that are part 
# of scikit-learn.
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, train_size=0.8)

Argue "for" or "against" splitting your data using an 80/20 split. That is, why is the 80/20 split appropriate (or not) for your dataset?  

In [80]:
### 2. Modeling

#### 2.1 One-Versus-All Logistic Regression Classifier

In [81]:
from scipy.special import expit
from sklearn.metrics import accuracy_score

class BinaryLogisticRegression:
    def __init__(self, eta, iterations=20, C=0.001):
        self.eta = eta
        self.iters = iterations
        self.C = C
        # internally we will store the weights as self.w_ to keep with sklearn conventions

    def __str__(self):
        if(hasattr(self,'w_')):
            return 'Binary Logistic Regression Object with coefficients:\n'+ str(self.w_) # is we have trained the object
        else:
            return 'Untrained Binary Logistic Regression Object'

    # convenience, private:
    @staticmethod
    def _add_bias(X):
        return np.hstack((np.ones((X.shape[0],1)),X)) # add bias term

    @staticmethod
    def _sigmoid(theta):
        # increase stability, redefine sigmoid operation
        return expit(theta) #1/(1+np.exp(-theta))

    # vectorized gradient calculation with regularization using L2 Norm
    def _get_gradient(self,X,y):
        ydiff = y-self.predict_proba(X,add_bias=False).ravel() # get y difference
        gradient = np.mean(X * ydiff[:,np.newaxis], axis=0) # make ydiff a column vector and multiply through

        gradient = gradient.reshape(self.w_.shape)
        gradient[1:] += -2 * self.w_[1:] * self.C

        return gradient

    # public:
    def predict_proba(self,X,add_bias=True):
        # add bias term if requested
        Xb = self._add_bias(X) if add_bias else X
        return self._sigmoid(Xb @ self.w_) # return the probability y=1

    def predict(self,X):
        return (self.predict_proba(X)>0.5) #return the actual prediction


    def fit(self, X, y):
        Xb = self._add_bias(X) # add bias term
        num_samples, num_features = Xb.shape

        self.w_ = np.zeros((num_features,1)) # init weight vector to zeros

        # for as many as the max iterations
        for _ in range(self.iters):
            gradient = self._get_gradient(Xb,y)
            self.w_ += gradient*self.eta # multiply by learning rate
            # add bacause maximizing

# for this, we won't perform our own BFGS implementation
# (it takes a fair amount of code and understanding, which we haven't setup yet)
# luckily for us, scipy has its own BFGS implementation:
from scipy.optimize import fmin_bfgs # maybe the most common bfgs algorithm in the world
from numpy import ma
class BFGSBinaryLogisticRegression(BinaryLogisticRegression):

    @staticmethod
    def objective_function(w,X,y,C):
        g = expit(X @ w)
        # invert this because scipy minimizes, but we derived all formulas for maximzing
        return -np.sum(ma.log(g[y==1]))-np.sum(ma.log(1-g[y==0])) + C*sum(w**2)
        #-np.sum(y*np.log(g)+(1-y)*np.log(1-g))

    @staticmethod
    def objective_gradient(w,X,y,C):
        g = expit(X @ w)
        ydiff = y-g # get y difference
        gradient = np.mean(X * ydiff[:,np.newaxis], axis=0)
        gradient = gradient.reshape(w.shape)
        gradient[1:] += -2 * w[1:] * C
        return -gradient

    # just overwrite fit function
    def fit(self, X, y):
        Xb = self._add_bias(X) # add bias term
        num_samples, num_features = Xb.shape

        self.w_ = fmin_bfgs(self.objective_function, # what to optimize
                            np.zeros((num_features,1)), # starting point
                            fprime=self.objective_gradient, # gradient function
                            args=(Xb,y,self.C), # extra args for gradient and objective function
                            gtol=1e-03, # stopping criteria for gradient, |v_k|
                            maxiter=self.iters, # stopping criteria iterations
                            disp=False)

        self.w_ = self.w_.reshape((num_features,1))

class StochasticLogisticRegression(BinaryLogisticRegression):
    # stochastic gradient calculation
    def _get_gradient(self,X,y):
        idx = int(np.random.rand()*len(y)) # grab random instance
        ydiff = y[idx]-self.predict_proba(X[idx],add_bias=False) # get y difference (now scalar)
        gradient = X[idx] * ydiff[:,np.newaxis] # make ydiff a column vector and multiply through

        gradient = gradient.reshape(self.w_.shape)
        gradient[1:] += -2 * self.w_[1:] * self.C

        return gradient

class MultiClassLogisticRegression:
    def __init__(self, eta, iterations=20,
                 C=0.0001,
                 solver=BFGSBinaryLogisticRegression):
        self.eta = eta
        self.iters = iterations
        self.C = C
        self.solver = solver
        self.classifiers_ = []
        # internally we will store the weights as self.w_ to keep with sklearn conventions

    def __str__(self):
        if(hasattr(self,'w_')):
            return 'MultiClass Logistic Regression Object with coefficients:\n'+ str(self.w_) # is we have trained the object
        else:
            return 'Untrained MultiClass Logistic Regression Object'

    def fit(self,X,y):
        num_samples, num_features = X.shape
        self.unique_ = np.sort(np.unique(y)) # get each unique class value
        num_unique_classes = len(self.unique_)
        self.classifiers_ = []
        for i,yval in enumerate(self.unique_): # for each unique value
            y_binary = np.array(y==yval).astype(int) # create a binary problem

            # train the binary classifier for this class

            hblr = self.solver(eta=self.eta,iterations=self.iters,C=self.C)
            hblr.fit(X,y_binary)

            # add the trained classifier to the list
            self.classifiers_.append(hblr)

        # save all the weights into one matrix, separate column for each class
        self.w_ = np.hstack([x.w_ for x in self.classifiers_]).T

    def predict_proba(self,X):
        probs = []
        for hblr in self.classifiers_:
            probs.append(hblr.predict_proba(X).reshape((len(X),1))) # get probability for each classifier

        return np.hstack(probs) # make into single matrix

    def predict(self,X):
        return self.unique_[np.argmax(self.predict_proba(X),axis=1)] # take argmax along row

In [82]:
%%time
lr = MultiClassLogisticRegression(eta=1,
                                  iterations=10,
                                  C=0.01,
                                  solver=BFGSBinaryLogisticRegression
                                  )
lr.fit(X,y)
print(lr)
yhat = lr.predict(X)
print('Accuracy of: ',accuracy_score(y,yhat))
unique_yhat, counts_yhat = np.unique(yhat, return_counts=True)
unique_y, counts_y = np.unique(y, return_counts=True)
print(np.asarray((unique_yhat, counts_yhat)).T)
print(np.asarray((unique_y, counts_y)).T)

MultiClass Logistic Regression Object with coefficients:
[[ 3.60041800e-02 -1.99950311e-01  1.99394084e-01 -7.00805609e-02
   1.10430479e-01  2.03350875e-02  2.39445418e-01 -1.15032110e-01
   1.88407317e-04  5.29461893e-02 -2.64533541e-02 -4.84603939e-01]
 [-1.33213685e-02 -2.02637710e-01  7.29978171e-02 -3.56067713e-02
   1.09315481e-01  9.42804505e-04 -2.76877509e-02 -6.19410921e-03
  -8.85414379e-03 -1.75117229e-03 -2.86013228e-02 -1.29969270e-01]
 [ 3.14378158e-01  8.62528254e-02  5.88964617e-01 -2.99659524e-01
  -4.23856598e-02  5.89540660e-02 -2.38097180e-02  2.24295164e-02
   2.82097604e-01  1.00630940e+00 -2.44107860e-01 -5.42092242e-01]
 [-1.08687196e-01 -3.29365462e-02 -3.04027108e-01  7.14684792e-02
  -5.10858077e-02 -2.18064724e-02  2.91166852e-02 -1.52508299e-02
  -9.63980049e-02 -3.33987229e-01  1.17367626e-01  1.53051874e-01]
 [-1.87695550e-01 -1.05963414e-01 -3.45554609e-01  2.00046599e-01
   8.59918596e-02 -3.43371948e-02  8.47716799e-03 -1.44051783e-02
  -1.67151665e-

In [83]:
%%time
from sklearn.linear_model import LogisticRegression as SKLogisticRegression

lr_sk = SKLogisticRegression(solver='liblinear') # all params default

lr_sk.fit(X,y)
print(np.hstack((lr_sk.intercept_[:,np.newaxis],lr_sk.coef_)))
yhat = lr_sk.predict(X)
print('Accuracy of: ',accuracy_score(y,yhat))

[[-3.37197495e-02  1.09708363e-01  2.42796660e+00 -2.23410823e-01
   1.64857120e-01  3.24699650e-01  7.55254563e-02 -7.95551123e-02
  -2.99806873e-02  6.21057154e-01 -3.82170117e-01 -7.79665183e-01]
 [-2.45158889e-01 -2.10323105e-01  2.41543387e+00 -8.33229223e-02
   1.75220599e-01  1.38489112e-01 -3.16965042e-02 -9.70928876e-03
  -2.49562731e-01  3.02405454e-01 -6.43140570e-01 -2.71117247e-01]
 [ 1.71160969e+00  2.71481781e-02  1.63716597e+00  3.70049307e-01
  -5.51552266e-02  1.42692953e+00 -2.19455524e-02  2.02353831e-02
   1.71287795e+00  1.33928851e+00 -1.74444792e+00 -8.53826116e-01]
 [-6.28216289e-01  2.51847309e-02 -1.43048098e+00 -9.43740602e-01
  -2.50081176e-02 -3.27507135e-01  2.45505399e-02 -1.31396676e-02
  -5.92353976e-01 -2.02069846e-01  7.59360059e-01  1.97688848e-01]
 [-1.41293663e+00 -4.15938505e-02 -2.88320725e+00  1.30452858e-01
   1.05686214e-01 -1.36620641e+00  1.42124707e-02 -1.66049125e-02
  -1.42182125e+00 -2.21190066e+00  1.94348355e+00  8.15806407e-01]
 [-6.

#### 2. Training Classifier for Good Generalization Performance

Is your method of selecting parameters justified? That is, do you think there is any "data snooping" involved with this method of selecting parameters?

#### 2.3 Comparing Best Performing Procedure to Scikit-Learn

In [84]:
# Visualize the performance differences in terms of training time and classification performance.

Discuss the results. 

### 3. Deployment

Which implementation of logistic regression would you advise be used in a deployed machine learning model, your implementation or scikit-learn (or other third party)? Why?

### 4. BFGS (Can change but thought this would be better)

In [85]:
# Implementation of BFGS

Compare your performance accuracy and runtime to the BFGS implementation in SciPy (that we used in lecture). 
